In [5]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

# Base URL
url = "https://www.yoox.com/us/men/shoponline/misbhv_d#"
#url = "https://www.yoox.com/us/men/shoponline/jil%20sander_d#" 
#url = "https://www.yoox.com/us/men/shoponline/jil%20sander_d#/d=77&dept=men&gender=U&page=2&season=X" 

headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0'
}

# Send a GET request to fetch the page content
response = requests.get(url, headers=headers, timeout=10)

if response.status_code == 200:
    print("response successfully retrieved")
else:
    print("failed to retrieve response")

response successfully retrieved


In [7]:
soup = BeautifulSoup(response.text, 'html.parser')
items = soup.find_all('div', class_='itemContainer')
df = pd.DataFrame(columns = ('link', 'brand', 'category', 'old_price', 'new_price', 'sizes'))

for item in items:
    item_data = item.find('div', class_='itemData')

    link = item_data.find('a', class_='itemlink')['href'] if item_data.find('a', class_='itemlink') else 'NA'
    brand = item_data.find('div', class_='brand').text.strip() if item_data.find('div', class_='brand') else 'NA'
    category = item_data.find('div', class_='microcategory').text.strip() if item_data.find('div', class_='microcategory') else 'NA'
    
    
    old_price = float(item_data.find('span', class_='oldprice').text.replace("$", "").replace(",", "").strip()) if item_data.find('span', class_='oldprice') else 'NA'
    new_price = float(item_data.find('div', class_='retail-newprice').text.replace("$", "").replace(",", "").strip()) if item_data.find('div', class_='retail-newprice') else 'NA'
    per_off = 1-new_price/old_price if (new_price != 'NA' and old_price != 'NA') else 0
    
    # if old_price and new_price are not available, the item is not discounted
    
    sizes_elements = item_data.find_all('span', class_='aSize')
    sizes = ', '.join([size.text for size in sizes_elements]) if sizes_elements else 'NA'

    new_row = {'link' : f'https://www.yoox.com{link}',
               'brand' : brand,
               'category' : category,
               'sizes' : sizes,
               'old_price' : old_price,
               'new_price' : new_price,
               'per_off' : per_off}
        
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

df.sort_values(by='per_off', ascending=False)
#df

,link,brand,category,old_price,new_price,sizes,per_off
5,https://www.yoox.com/us/11882986GI/item#dept=m...,MISBHV,Sneakers,615.0,118.0,"7, 8, 10",0.808130
1,https://www.yoox.com/us/17632867WF/item#dept=m...,MISBHV,Sneakers,490.0,117.0,11,0.761224
6,https://www.yoox.com/us/45801101DU/item#dept=m...,MISBHV,Handbags,676.0,200.0,NA,0.704142
4,https://www.yoox.com/us/17288428KF/item#dept=m...,MISBHV,Sneakers,557.0,169.0,8,0.696589
14,https://www.yoox.com/us/49859647GD/item#dept=m...,MISBHV,Blazers,850.0,310.0,M,0.635294
0,https://www.yoox.com/us/17625145AN/item#dept=m...,MISBHV,Sneakers,490.0,216.0,"7, 8",0.559184
2,https://www.yoox.com/us/17442883RD/item#dept=m...,MISBHV,Sneakers,299.0,136.0,5,0.545151
7,https://www.yoox.com/us/12805908VH/item#dept=m...,MISBHV,Solid color shirts,401.0,192.0,S,0.521197
3,https://www.yoox.com/us/17218202VW/item#dept=m...,MISBHV,Sneakers,475.0,245.0,8,0.484211
9,https://www.yoox.com/us/17315245WL/item#dept=m...,MISBHV,Loafers,489.0,271.0,8,0.445808


## Try a Selenium solution...

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
import pandas as pd
import csv
import time

In [ ]:
url = "https://www.yoox.com/us/men/shoponline/jil%20sander_d#"
driver_path = r"C:\browser_automation\chromedriver-win64\chromedriver.exe"

# Setup Selenium WebDriver
service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service)
driver.get(url)

# DataFrame to hold the scraped data
df = pd.DataFrame(columns=['link', 'brand', 'category', 'old_price', 'new_price', 'sizes'])

def scrape_current_page():
    items = driver.find_elements(By.CLASS_NAME, 'itemContainer')
    for i in range(len(items)):
        # Re-fetch the items on the page to avoid stale references
        item = driver.find_elements(By.CLASS_NAME, 'itemContainer')[i]
        
        try:
            link_element = item.find_element(By.CSS_SELECTOR, 'a.itemlink')
            link = link_element.get_attribute('href')
        except NoSuchElementException:
            link = 'NA'
        
        try:
            brand = item.find_element(By.CLASS_NAME, 'brand').text
        except NoSuchElementException:
            brand = 'NA'
        
        try:
            category = item.find_element(By.CLASS_NAME, 'microcategory').text
        except NoSuchElementException:
            category = 'NA'
        
        try:
            old_price = float(item.find_element(By.CLASS_NAME, 'oldprice').text.replace("$", "").replace(",", "").strip())
        except NoSuchElementException:
            old_price = 'NA'
        
        try:
            new_price = float(item.find_element(By.CLASS_NAME, 'retail-newprice').text.replace("$", "").replace(",", "").strip())
        except NoSuchElementException:
            new_price = 'NA'
        
        per_off = 1-new_price/old_price if (new_price != 'NA' and old_price != 'NA') else 0
        
        try:
            sizes_elements = item.find_elements(By.CLASS_NAME, 'aSize')
            sizes = ', '.join([size.text for size in sizes_elements])
        except NoSuchElementException:
            sizes = 'NA'

        new_row = {'link': link, 'brand': brand, 'category': category, 'old_price': old_price, 'new_price': new_price, 'sizes': sizes, 'per_off': per_off}
        global df
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


while True:
    scrape_current_page()
    try:
        next_button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CLASS_NAME, 'nextPage')))
        driver.execute_script("arguments[0].click();", next_button)
        WebDriverWait(driver, 2).until(EC.staleness_of(next_button))
    except (TimeoutException, NoSuchElementException):
        break

driver.quit()

In [ ]:
df.sort_values(by='per_off', ascending=False)

### Write to csv

In [10]:
url = "https://www.yoox.com/us/men/shoponline/"
url = "https://www.yoox.com/us/men/shoponline/jeans%20and%20denim_mc#/dept=clothingmen&gender=U&page=1&attributes=%7b%27ctgr%27%3a%5b%27jns%27%5d%2c%27smrtfnd%27%3a%5b%27smbstndr%27%2c%27npcnsl%27%2c%27smjstdscnt%27%2c%27smtpdsgnrs%27%5d%7d&season=X"

driver_path = r"C:\browser_automation\chromedriver-win64\chromedriver.exe"

# Setup Selenium WebDriver
service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service)
driver.get(url)


with open('scraped_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['link', 'brand', 'category', 'old_price', 'new_price', 'sizes', 'per_off'])
    writer.writeheader()

    def scrape_current_page():
        items = driver.find_elements(By.CLASS_NAME, 'itemContainer')
        for i in range(len(items)):
            item = driver.find_elements(By.CLASS_NAME, 'itemContainer')[i]
            
            try:
                link_element = item.find_element(By.CSS_SELECTOR, 'a.itemlink')
                link = link_element.get_attribute('href')
            except NoSuchElementException:
                link = 'NA'
            
            try:
                brand = item.find_element(By.CLASS_NAME, 'brand').text
            except NoSuchElementException:
                brand = 'NA'
            
            try:
                category = item.find_element(By.CLASS_NAME, 'microcategory').text
            except NoSuchElementException:
                category = 'NA'
            
            try:
                old_price = float(item.find_element(By.CLASS_NAME, 'oldprice').text.replace("$", "").replace(",", "").strip())
            except NoSuchElementException:
                old_price = 'NA'
            
            try:
                new_price = float(item.find_element(By.CLASS_NAME, 'retail-newprice').text.replace("$", "").replace(",", "").strip())
            except NoSuchElementException:
                new_price = 'NA'
            
            per_off = 1-new_price/old_price if (new_price != 'NA' and old_price != 'NA') else 0
            
            try:
                sizes_elements = item.find_elements(By.CLASS_NAME, 'aSize')
                sizes = ', '.join([size.text for size in sizes_elements])
            except NoSuchElementException:
                sizes = 'NA'

            # Append data to the CSV file
            writer.writerow({'link': link, 'brand': brand, 'category': category, 'old_price': old_price, 'new_price': new_price, 'sizes': sizes, 'per_off': per_off})

    while True:
        scrape_current_page()
        try:
            next_button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CLASS_NAME, 'nextPage')))
            driver.execute_script("arguments[0].click();", next_button)
            WebDriverWait(driver, 2).until(EC.staleness_of(next_button))
        except (TimeoutException, NoSuchElementException):
            break

driver.quit()

In [9]:
driver.quit()